In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [23]:
li = []
for n in range(10):    
    df = pd.read_csv(f'data/{n}.csv')
    li.append(df)
data = pd.concat(li, axis=0, ignore_index=True)
data.shape

(8833, 287)

In [24]:
data.head()

,Time,Flow rate,Zone_ids,Zone2_K1,Zone2_K2,Zone2_K3,Zone2_Pressure,Zone2_tfilling,Zone2_VoidContent,Zone2_VoidQuality,...,Zone31_tfilling,Zone31_VoidContent,Zone31_VoidQuality,Zone31_Filling,Zone31_FillingQuality,Total_PorosityQuantity,Total_PorosityQuality,Total_UnfilledZones,Total_FillingQuality,TOTAL_QUALITY
0,0.065529,6.727330e-07,2,4.000000e-11,5.000000e-11,2.000000e-12,0.0,7.04083,0.51306,1.0,...,7.15607,0.430224,1.0,1.0,1.0,0,1,0,1,1
1,0.094341,7.170040e-07,2,4.000000e-11,5.000000e-11,2.000000e-12,0.0,7.04083,0.51306,1.0,...,7.15607,0.430224,1.0,1.0,1.0,0,1,0,1,1
2,0.116504,7.614100e-07,2,4.000000e-11,5.000000e-11,2.000000e-12,0.0,7.04083,0.51306,1.0,...,7.15607,0.430224,1.0,1.0,1.0,0,1,0,1,1
3,0.165700,8.404510e-07,2,4.000000e-11,5.000000e-11,2.000000e-12,0.0,7.04083,0.51306,1.0,...,7.15607,0.430224,1.0,1.0,1.0,0,1,0,1,1
4,0.186822,8.817000e-07,2,4.000000e-11,5.000000e-11,2.000000e-12,0.0,7.04083,0.51306,1.0,...,7.15607,0.430224,1.0,1.0,1.0,0,1,0,1,1


In [42]:
total = ['Total_PorosityQuantity', 'Total_PorosityQuality', 'Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY']
X = data.drop(axis=1, columns=total)
Y = data[total]
print(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

      Total_PorosityQuantity  Total_PorosityQuality  Total_UnfilledZones  \
0                          0                      1                    0   
1                          0                      1                    0   
2                          0                      1                    0   
3                          0                      1                    0   
4                          0                      1                    0   
...                      ...                    ...                  ...   
8828                       0                      1                    1   
8829                       0                      1                    1   
8830                       0                      1                    1   
8831                       0                      1                    1   
8832                       0                      1                    1   

      Total_FillingQuality  TOTAL_QUALITY  
0                        1              1  

In [39]:
rf_model = RandomForestClassifier(class_weight="balanced", n_estimators=100).fit(X_train, y_train)

In [40]:
y_pred = rf_model.predict(X_test)
accuracy_score(y_test, y_pred)

1.0